# Finding similar expenses
 
Reading these features:
* Name;
* Category (can be multiple, if separated with comma);
* Cost;
* Date (year-month-day);
* Weekday;

This notebook wishes to find the expenses most similar to another. 

Additional Parameters:
* ignoreEqualNames -> when searching similar expense for an expense of name X, never takes an expense with the same name (even if in a different date). This can be useful for finding similar expenses from a recurrent expense, ignoring all of its previous ;
* ignoreAllEqualNames -> when searching similar expenses, if an expense Y was found for expense X, no more expenses with the same name as Y can be compared with X;


# Method

Currently I will be using cosine_similarity, because its the most simple strategy. 

# Reading example

In [8]:
!pip install pandas
!pip install openpyxl
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\Emily\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\Emily\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 330.3 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.8 kB 262.6 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 293.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.1 MB 1.9 MB/s eta 0:00:06
   ---------------------------------------- 0.1/11.1 MB 1.7 MB/s eta 0:00:07
   ---------------------------------------- 0.1/11.1 MB 737.3 kB/s eta 0:00:15
    --------------------------------------- 0.2/11.1 MB 1.1 MB/s eta 0:00:10
   - -------------------------------------- 0.3/11.1 MB 1.5 MB/s eta 0:00:08
   - -------------------------------------- 0.3/11.1 MB 1.2 MB/s eta 0:00:10
 


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\Emily\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:

import pandas as pd

dt = pd.read_excel("nlp_example.xlsx")


dt.head()

,name,category,cost,date,weekday
0,Condomínio - fundo reserva,"Moradia, Fixas",5.00,2025-02-05,Wednesday
1,Air fryer,"Moradia, Fixas",200.00,2025-02-05,Wednesday
2,Transporte Atacadão ida e volta,"Transporte, Uber",19.29,2025-02-03,Monday
3,Container energetic,"Alimentação, Lanche",9.00,2025-02-01,Saturday
4,RU,"Moradia, Fixas",3.00,2025-02-01,Saturday


In [13]:


from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

clean_dt = dt.dropna()

# Name wont be used to compare similarity for now
clean_dt = clean_dt.drop(columns=[  "weekday"])

all_categories = clean_dt["category"].apply(lambda x: x.split(", ")).explode().unique() 
 
print("all categories: ", all_categories)
clean_dt["category"] = clean_dt["category"].apply(lambda x: x.split(", "))#.apply(lambda x: (all_categories.tolist().index(i) for i in x))
clean_dt["category"] = clean_dt["category"].apply(lambda x: [all_categories.tolist().index(i) for i in x])
clean_dt["category"] = clean_dt["category"].apply(lambda x: [1 if i in x else 0 for i in range(len(all_categories))])
clean_dt["date"] = pd.to_datetime(clean_dt["date"])

# Get the first date as reference
reference_date = clean_dt["date"].min()

# Compute days since reference date
clean_dt["days_since"] = (clean_dt["date"] - reference_date).dt.days
clean_dt = clean_dt.drop(columns=["date"])

clean_dt.head()

all categories:  ['Moradia' 'Fixas' 'Transporte' 'Uber' 'Alimentação' 'Lanche' 'Almoço'
 'Vestuário' 'Pagamentos']


,name,category,cost,days_since
0,Condomínio - fundo reserva,"[1, 1, 0, 0, 0, 0, 0, 0, 0]",5.00,4
1,Air fryer,"[1, 1, 0, 0, 0, 0, 0, 0, 0]",200.00,4
2,Transporte Atacadão ida e volta,"[0, 0, 1, 1, 0, 0, 0, 0, 0]",19.29,2
3,Container energetic,"[0, 0, 0, 0, 1, 1, 0, 0, 0]",9.00,0
4,RU,"[1, 1, 0, 0, 0, 0, 0, 0, 0]",3.00,0


In [19]:
import numpy as np
rows = clean_dt.shape[0]
cosine_sim = []
  
all_values = [np.array(v).reshape(1, -1)[0][1:] for v in clean_dt.values]

all_values = [ np.hstack([x if not isinstance(x, list) else np.array(x) for x in arr]) for arr in all_values]

for i in range(rows):
    
    first_value = clean_dt.iloc[i].values
    
    first_value = np.array(first_value).reshape(1, -1)[0]
    first_value =  np.hstack([x if not isinstance(x, list) else np.array(x) for x in first_value])
    similarity = cosine_similarity([first_value[1:]], all_values)
    similarity[0][i] = 0
    cosine_sim.append(similarity[0])
    '''
    for j in range(rows):
        if i == j:
            cosine_sim.append(0)
            continue
        second_value = clean_dt.iloc[j].values
        second_value = np.array(second_value).reshape(1, -1)[0]
        cosine_sim.append(cosine_similarity([first_value[1:]], [second_value[1:]])[0][0])
    '''

print("SIMILARITIES")
for i in range(rows):
    
    highest_similarity = max(cosine_sim[i])
    index = np.where(cosine_sim[i] == highest_similarity)[0] 
    index = int(index)
  
    print(f"{clean_dt.iloc[i].values[0]}", end=" ")
    print(f"-> {clean_dt.iloc[index].values[0]}", end=" ")
    print(f"with {highest_similarity.round(2)} similarity")
    print("The average similarity is: ", np.mean(cosine_sim[i]))

SIMILARITIES
Condomínio - fundo reserva -> Roupa Pompéi para Anna with 0.91 similarity
The average similarity is:  0.7232026976460374
Air fryer -> Celular with 1.0 similarity
The average similarity is:  0.8944963014255635
Transporte Atacadão ida e volta -> Cama with 1.0 similarity
The average similarity is:  0.8936884008908769
Container energetic -> Container hambúrguer e energético with 1.0 similarity
The average similarity is:  0.8860082455771696
RU -> Cartão alimentação with 0.92 similarity
The average similarity is:  0.8217762816305344
RU -> Container energetic with 0.93 similarity
The average similarity is:  0.8172300225552594
Container hambúrguer e energético -> Aluguel with 1.0 similarity
The average similarity is:  0.8923776509493155
Rancho -> Aluguel with 1.0 similarity
The average similarity is:  0.89312218140776
Cama -> Air fryer with 1.0 similarity
The average similarity is:  0.8963055240106251
Roupa Pompéi para Anna -> Transporte Atacadão ida e volta with 0.98 similarity
T

C:\Users\Emily\AppData\Local\Temp\ipykernel_12496\1129621982.py:33: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  index = int(index)


# Transformer 

In [6]:
 
import pandas as pd
import re
from datetime import datetime
#from FlagEmbedding import BGEM3FlagModel
# Use a pipeline as a high-level helper
#from transformers import pipeline
#from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer, util


############ Functions ############
 
 

#@st.cache_resource
def calculate_similarities(all_data, target_to_compare):
   
    
    # transform all the column Summary into strings, as a list, obeying the order of the table
    
    all_data_str = []

    for i in range(len(all_data)):
        all_data_str.append(str(all_data.iloc[i]))

    # run the model

    target_to_compare_str = str(target_to_compare)

    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    #Compute embedding for both lists
    embedding_1= model.encode(target_to_compare_str, convert_to_tensor=True)
    embedding_2 = model.encode(all_data_str, convert_to_tensor=True)

    similarity = util.pytorch_cos_sim(embedding_1, embedding_2)

    
    # place it back in the dataset
    all_data.loc[:,'similarity'] = similarity[0].tolist()

    #all_data['summary_fixed'] = all_data.Summary.str.replace(r'[\n\s]{2,}', ' ', regex=True)
 
    return all_data.sort_values(by='similarity', ascending=False)#[:5]
    


############  ############

for i in range(dt.shape[0]):
    
    value = dt.iloc[i]
    all_others = dt.drop(i)
    similar = calculate_similarities(all_others, value)
    
    print(f"Values similar to {value}")
    print(similar)

Values similar to name        Condomínio - fundo reserva
category                Moradia, Fixas
cost                               5.0
date               2025-02-05 00:00:00
weekday                      Wednesday
Name: 0, dtype: object
                                 name             category    cost       date  \
12                            Aluguel       Moradia, Fixas  900.00 2025-02-04   
4                                  RU       Moradia, Fixas    3.00 2025-02-01   
8                                Cama       Moradia, Fixas  156.83 2025-02-11   
13                 Cartão alimentação       Moradia, Fixas   50.00 2025-02-01   
5                                  RU  Alimentação, Almoço    3.00 2025-02-01   
9              Roupa Pompéi para Anna            Vestuário   31.74 2025-02-11   
7                              Rancho       Moradia, Fixas  409.25 2025-02-02   
6   Container hambúrguer e energético  Alimentação, Lanche   18.48 2025-02-01   
1                           Air fry

# Características Derivadas

Através das informações iniciais, podemos derivar outras características binárias. A seguir, as possíveis características derivadas:

* (✔️) isWeekend -> ;
* (✔️) isGlobalExpensive;
* (❌)